# Applications

In [10]:
def json_print(data):
    print(json.dumps(data, indent=2))

In [11]:
import weaviate, os, json
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

auth_config = weaviate.auth.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY"))

client = weaviate.Client(
    url=os.getenv("WEAVIATE_API_URL"),
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
        "X-Cohere-BaseURL": os.getenv("CO_API_URL")
    }
)

client.is_ready() #check if True

True

### 2. How many vectors are stored in this database

In [12]:
print(json.dumps(client.query.aggregate("Wikipedia").with_meta_count().do(), indent=2))

{

  "data": {

    "Aggregate": {

      "Wikipedia": [

        {

          "meta": {

            "count": 4306800

          }

        }

      ]

    }

  }

}


### 3. Perform search over them to find concepts you are interested in!

In [13]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_limit(5)
            .do()
           )

json_print(response)

{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "lang": "en",

          "text": "Many locals and tourists frequent the Southern California coast for its beaches. Some of southern California's most popular beaches are Malibu, Laguna Beach, La Jolla, Manhattan Beach, and Hermosa Beach. Southern California is also known for its mountain resort communities, such as Big Bear Lake, Lake Arrowhead, and Wrightwood, and their ski resorts, like Bear Mountain, Snow Summit, Snow Valley Mountain Resort, and Mountain High. The inland desert city of Palm Springs is also popular.",

          "title": "Southern California",

          "url": "https://en.wikipedia.org/wiki?curid=62520",

          "views": 2000

        },

        {

          "lang": "en",

          "text": "Carbon Beach, Surfrider Beach, Westward Beach, Escondido Beach, Paradise Cove, Point Dume, Pirates Cove, Zuma Beach, Trancas and Encinal Bluffs are places along the coast in Malibu. Point Dume forms the

In [14]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "lang": "en",

          "text": "Many locals and tourists frequent the Southern California coast for its beaches. Some of southern California's most popular beaches are Malibu, Laguna Beach, La Jolla, Manhattan Beach, and Hermosa Beach. Southern California is also known for its mountain resort communities, such as Big Bear Lake, Lake Arrowhead, and Wrightwood, and their ski resorts, like Bear Mountain, Snow Summit, Snow Valley Mountain Resort, and Mountain High. The inland desert city of Palm Springs is also popular.",

          "title": "Southern California",

          "url": "https://en.wikipedia.org/wiki?curid=62520",

          "views": 2000

        },

        {

          "lang": "en",

          "text": "Carbon Beach, Surfrider Beach, Westward Beach, Escondido Beach, Paradise Cove, Point Dume, Pirates Cove, Zuma Beach, Trancas and Encinal Bluffs are places along the coast in Malibu. Point Dume forms the

In [16]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Miejsca na wakacje w Kalifornii"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "lang": "en",

          "text": "Many locals and tourists frequent the Southern California coast for its beaches. Some of southern California's most popular beaches are Malibu, Laguna Beach, La Jolla, Manhattan Beach, and Hermosa Beach. Southern California is also known for its mountain resort communities, such as Big Bear Lake, Lake Arrowhead, and Wrightwood, and their ski resorts, like Bear Mountain, Snow Summit, Snow Valley Mountain Resort, and Mountain High. The inland desert city of Palm Springs is also popular.",

          "title": "Southern California",

          "url": "https://en.wikipedia.org/wiki?curid=62520",

          "views": 2000

        },

        {

          "lang": "en",

          "text": "Malibu has been used as a location or setting for many films, television programs, fashion shoots and music videos.",

          "title": "Malibu, California",

          "url": "https://en.wikipedia.or

In [17]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "أماكن العطلات في كاليفورنيا"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "lang": "en",

          "text": "Many locals and tourists frequent the Southern California coast for its beaches. Some of southern California's most popular beaches are Malibu, Laguna Beach, La Jolla, Manhattan Beach, and Hermosa Beach. Southern California is also known for its mountain resort communities, such as Big Bear Lake, Lake Arrowhead, and Wrightwood, and their ski resorts, like Bear Mountain, Snow Summit, Snow Valley Mountain Resort, and Mountain High. The inland desert city of Palm Springs is also popular.",

          "title": "Southern California",

          "url": "https://en.wikipedia.org/wiki?curid=62520",

          "views": 2000

        },

        {

          "lang": "en",

          "text": "The park is located east of Webb Way, and between Civic Center Way on the north and PCH to the south. It was the site of the annual Labor Day Weekend Kiwanis Club Chili Cook-Off from 1982 to 2009 (in 20

## Retrieval Augmented Generation

### Single Prompt

In [18]:
prompt = "Write me a facebook ad about {title} using information inside {text}"
result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(single_prompt=prompt)
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)

{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "_additional": {

            "generate": {

              "error": null,

              "singleResult": "Looking for a perfect vacation spot with a little bit of everything? Look no further than Southern California! From the beautiful beaches like Malibu, Laguna Beach, and La Jolla to the incredible ski resorts of Bear Mountain and Mountain High, there's something for everyone here! \n\nIf you want a more low-key, relaxed day, head to the desert city of Palm Springs for a peaceful, yet fun, time. Or head to the beach for a surf and some sun - there are plenty to choose from! \n\nThe SoCal region has so much to offer - you won't run out of things to do and see. Come experience the beauty and diversity of Southern California today! #SoCal #Travel #Vacation #Beaches #Mountains #DesertAdventures"

            }

          },

          "text": "Many locals and tourists frequent the Southern California coast for its b

### Group Task

In [19]:
generate_prompt = "Summarize what these posts are about in two paragraphs."

result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(grouped_task=generate_prompt) # Pass in all objects at once
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)


{

  "data": {

    "Get": {

      "Wikipedia": [

        {

          "_additional": {

            "generate": {

              "error": null,

              "groupedResult": "The first post is about Southern California and discusses popular beaches like Malibu, Laguna Beach, and Manhattan Beach as well as mountain resort communities such as Big Bear Lake and Lake Arrowhead. The post also mentions the inland desert city of Palm Springs which is also a popular destination. \n\nThe second post is about Malibu, California and lists several beaches found along the coast including Carbon Beach, Surfrider Beach, and Zuma Beach. The post also mentions Point Dume which forms the northern end of Santa Monica Bay. Pirates Cove, directly below Point Dume, was previously known as a nude beach but is now illegal and subject to fines or arrest."

            }

          },

          "text": "Many locals and tourists frequent the Southern California coast for its beaches. Some of southern Calif